In [25]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import os
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline

In [26]:
pd.set_option('display.max_columns', None)

In [27]:
# Removendo linhas duplicadas

class RemoveDuplicates(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        print("Removendo duplicatas do DataFrame")
        linhas_inicio = X.shape[0]
        X_copy = X.copy()
        X_copy.drop_duplicates(inplace=True)
        linhas_fim = X_copy.shape[0]
        print(f"Tratamento de duplicatas concluído. Foram removidas {linhas_inicio - linhas_fim} linhas.")
        print()
        return X_copy

In [28]:
# Removendo linhas em que o pedido foi cancelado ou que possuem produtos sem dimensão

class RemoveInvalidRows(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.columns_to_check = ["order_approved_at", "product_height_cm"]
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        initial_rows = X.shape[0]
        X_copy = X.dropna(subset=self.columns_to_check)
        removed_rows = initial_rows - X_copy.shape[0]
        print(f"Removidas {removed_rows} linhas devido a valores nulos nas colunas: {', '.join(self.columns_to_check)}.")
        print()
        return X_copy


In [29]:
# Removendo colunas que não fornecem informações valiosas para o nosso modelo 
## ("'order_id.1', 'customer_id.1', 'product_name_lenght', 'product_description_lenght', 'product_photos_qty")

# Removendo colunas de localização, pois possuem muitos dados faltantes, sendo que já temos as informações de zip code, suficientes para o nosso modelo
## ('customer_city', 'customer_state', 'seller_city', 'seller_state')

class CleanColumns(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        columns_to_remove = [
            'product_name_lenght', 'product_description_lenght', 'product_photos_qty',
            'customer_city', 'customer_state', 'seller_city', 'seller_state'
        ]
        print("Removendo colunas específicas do DataFrame")
        X_copy = X.copy()
        X_copy.drop(columns=columns_to_remove, inplace=True)
        print(f"Colunas removidas: {columns_to_remove}")
        print()
        return X_copy

In [30]:
# Convertendo colunas de data para a formatação DateTime

class ConvertToDateTime(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.columns = [
            "order_purchase_timestamp",
            "order_approved_at",
            "order_delivered_carrier_date",
            "order_delivered_customer_date"
        ]
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        X_copy = X.copy()
        for column in self.columns:
            if column in X_copy.columns:
                print(f"Convertendo coluna {column} para datetime")
                X_copy[column] = pd.to_datetime(X_copy[column], errors='coerce')
        print()
        return X_copy

In [31]:
# Criando novas colunas para analisar o intervalo de tempo dedicado a cada etapa do processo

class AddTimeAnalysisColumns(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        X_copy = X.copy()

        # Calcular tempo de pedido em aprovação
        PEDIDO_APROVACAO = X_copy["order_approved_at"] - X_copy["order_purchase_timestamp"]
        PA2 = [(elem.total_seconds() / (24 * 3600)) if pd.notnull(elem) else None for elem in PEDIDO_APROVACAO]
        PA2 = [round(elem, 2) if elem is not None else None for elem in PA2]

        # Calcular tempo em que o pedido está sendo separado e enviado para a transportadora
        SEPARANDO_PEDIDO = X_copy["order_delivered_carrier_date"] - X_copy["order_approved_at"]
        SP2 = [(elem.total_seconds() / (24 * 3600)) if pd.notnull(elem) else None for elem in SEPARANDO_PEDIDO]
        SP2 = [round(elem, 2) if elem is not None else None for elem in SP2]

        # Calcular o tempo em que o pedido está em transporte até chegar na casa do cliente
        PEDIDO_TRANSPORTE = X_copy["order_delivered_customer_date"] - X_copy["order_delivered_carrier_date"]
        PT2 = [(elem.total_seconds() / (24 * 3600)) if pd.notnull(elem) else None for elem in PEDIDO_TRANSPORTE]
        PT2 = [round(elem, 2) if elem is not None else None for elem in PT2]

        # Inserir novas colunas no DataFrame
        X_copy.insert(4, "Pedido em aprovação", PA2)
        X_copy.insert(6, "Separando o pedido", SP2)
        X_copy.insert(8, "Pedido em transporte", PT2)

        return X_copy

In [32]:
# Removendo os casos em que o intervalo de tempo de determinada etapa é negativo

class RemoveNegatives(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.columns_to_treat = ["Pedido em aprovação", "Separando o pedido", "Pedido em transporte"]
        self.linhas_removidas = {}
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        X_copy = X.copy()
        for coluna in self.columns_to_treat:
            linhas_inicio = X_copy.shape[0]
            X_copy = X_copy[X_copy[coluna] >= 0]  
            self.linhas_removidas[coluna] = linhas_inicio - X_copy.shape[0]
        
        for coluna, num_linhas in self.linhas_removidas.items():
            print(f"Tratamento de números negativos concluído. Foram removidas {num_linhas} linhas devido à coluna '{coluna}'.")
        print()

        return X_copy

In [33]:
# Retirando outliers superiores

class OutliersTreatment(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.columns_to_treat = ["delivery_time", "Pedido em aprovação", "Separando o pedido", "Pedido em transporte"]
        self.outliers_limits = {}
        self.removed_lines = {}
    
    def fit(self, X, y=None):
        for coluna in self.columns_to_treat:
            q1 = np.percentile(X[coluna], 25)
            q3 = np.percentile(X[coluna], 75)
            iqr = q3 - q1
            out_sup = q3 + iqr * 1.5
            out_inf = q1 - iqr * 1.5
            self.outliers_limits[coluna] = (out_inf, out_sup)
        return self
    
    def transform(self, X):
        X_copy = X.copy()
        self.removed_lines = {}
        for coluna in self.columns_to_treat:
            out_inf, out_sup = self.outliers_limits[coluna]
            initial_rows = X_copy.shape[0]
            X_copy = X_copy[X_copy[coluna] < out_sup]
            removed = initial_rows - X_copy.shape[0]
            self.removed_lines[coluna] = removed
            print(f"Tratamento de outliers para coluna '{coluna}' concluído. Foram removidas {removed} linhas.")
        print()
        return X_copy


In [34]:
# Agrupando categorias de produtos 

class AgruparCategorias(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.categorias_a_agrupar = {
            'construcao_ferramentas_ferramentas': 'construcao_ferramentas',
            'construcao_ferramentas_construcao': 'construcao_ferramentas',
            'construcao_ferramentas_jardim': 'construcao_ferramentas',
            'construcao_ferramentas_iluminacao': 'construcao_ferramentas',
            'construcao_ferramentas_seguranca': 'construcao_ferramentas',
            'ferramentas_jardim': 'construcao_ferramentas',

            'moveis_sala': 'moveis',
            'moveis_quarto': 'moveis',
            'moveis_colchao_e_estofado': 'moveis',
            'moveis_cozinha_area_de_servico_jantar_e_jardim': 'moveis',
            'moveis_decoracao': 'moveis',
            'moveis_escritorio': 'moveis',

            'pc_gamer': 'pcs',

            'artes_e_artesanato': 'artes',

            'telefonia_fixa': 'telefonia',

            'alimentos': 'alimentos_bebidas',
            'bebidas': 'alimentos_bebidas',

            'cds_dvds_musicais': 'cds_dvds',
            'dvds_blu_ray': 'cds_dvds',

            'portateis_casa_forno_e_cafe': 'eletroportateis',

            'casa_conforto_2': 'casa_conforto',

            'eletrodomesticos_2': 'eletrodomesticos',

            'malas_acessorios': 'fashion',
            'fashion_bolsas_e_acessorios': 'fashion',
            'fashion_calcados': 'fashion',
            'fashion_underwear_e_moda_praia': 'fashion',
            'fashion_roupa_masculina': 'fashion',
            'fashion_esporte': 'fashion',
            'fashion_roupa_feminina': 'fashion',
            'fashion_roupa_infanto_juvenil': 'fashion',

            'eletronicos': 'informatica_acessorios',
            'tablets_impressao_imagem': 'informatica_acessorios',

            'la_cuisine': 'utilidades_domesticas',
            
            'fraldas_higiene': 'bebes'
        }
        self.num_modified = 0
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        X_copy = X.copy()
        modified_rows = X_copy['product_category_name'].replace(self.categorias_a_agrupar)
        self.num_modified = (X_copy['product_category_name'] != modified_rows).sum()
        X_copy['product_category_name'] = modified_rows
        print(f"Agrupamento de categorias concluído. Foram modificadas {self.num_modified} linhas.")
        return X_copy


In [35]:
pipeline_preprocessamento = Pipeline([
    ('Remover duplicatas', RemoveDuplicates()),
    ('Remover linhas nulas', RemoveInvalidRows()),
    ('Tirar colunas desnecessárias', CleanColumns()),
    ('Converter colunas para DateTime', ConvertToDateTime()),
    ('Criando colunas com intervalos de tempo', AddTimeAnalysisColumns()),
    ('Remover intervalos de tempo negativos', RemoveNegatives()),
    ('Remover outliers', OutliersTreatment()),
    ('Agrupar categorias de produtos', AgruparCategorias())
])

df = pd.read_csv("Data/DE&CO_14_06.csv")

dados_preprocessados = pipeline_preprocessamento.fit_transform(df)


Removendo duplicatas do DataFrame
Tratamento de duplicatas concluído. Foram removidas 7413 linhas.

Removidas 32 linhas devido a valores nulos nas colunas: order_approved_at, product_height_cm.

Removendo colunas específicas do DataFrame
Colunas removidas: ['product_name_lenght', 'product_description_lenght', 'product_photos_qty', 'customer_city', 'customer_state', 'seller_city', 'seller_state']

Convertendo coluna order_purchase_timestamp para datetime
Convertendo coluna order_approved_at para datetime
Convertendo coluna order_delivered_carrier_date para datetime
Convertendo coluna order_delivered_customer_date para datetime

Tratamento de números negativos concluído. Foram removidas 0 linhas devido à coluna 'Pedido em aprovação'.
Tratamento de números negativos concluído. Foram removidas 390 linhas devido à coluna 'Separando o pedido'.
Tratamento de números negativos concluído. Foram removidas 49 linhas devido à coluna 'Pedido em transporte'.

Tratamento de outliers para coluna 'deli

In [36]:
# Definindo o caminho para a pasta Data
current_dir = os.path.dirname(os.path.abspath('__file__'))
data_dir = os.path.join(current_dir, 'Data')


# Caminho completo para o arquivo CSV
csv_path = os.path.join(data_dir, 'Dataframelimpa_sem_latlong.csv')


# Exportando o DataFrame para um arquivo CSV
dados_preprocessados.to_csv(csv_path, index=False)